In [ ]:
import os
import sys
from time import sleep

import GCode
import GRBL
import numpy as np

# Draw a Brick Pattern

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

Attempt at programmatically making a brick pattern. 

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)



In [1]:
BrickHeight = 65
BrickLength = 225
BrickDepth = 102.5

BrickRatio = 215 / 65
BrickRatio


3.3076923076923075

In [2]:
BlockHeight = 89.0
BlockLength = 176.0


In [3]:
N_BrickRows = 5


In [4]:
H_Block_Brick = BlockHeight / N_BrickRows


In [5]:
L_Block_Brick = H_Block_Brick * BrickRatio


In [6]:
H_Block_Brick


17.8

In [7]:
L_Block_Brick


58.87692307692308

# Code:

In [9]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


ok


In [ ]:
cnc.home()


In [19]:
print(cnc.laser_mode)


1.0


In [20]:
cnc.cmd("G28")


['ok', 'ok']

In [21]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=200)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0, Z=0)
    return program


In [22]:
np.arange(H_Block_Brick, BlockHeight, H_Block_Brick)


array([17.8, 35.6, 53.4, 71.2])

In [23]:
Brick_Line = np.array([[0, H_Block_Brick], [BlockLength, H_Block_Brick]])
Brick_Line


array([[  0. ,  17.8],
       [176. ,  17.8]])

In [24]:
Brick_Line.shape[0]


2

In [25]:
for row_idx in range(Brick_Line.shape[0]):
    print(Brick_Line[:, row_idx])
    break


[  0. 176.]


In [26]:
default_line = np.array([[0, 0], [10, 5]])


In [27]:
default_line


array([[ 0,  0],
       [10,  5]])

Draw a line from (0, 0) to (10, 5).

In [28]:
prog = GCode.GCode()


def line(points=default_line, feed=300, power=150, dynamic_power=True):
    prog.G0(X=points[:, 0][0], Y=points[:, 0][1])


line()
prog


<GCode>[cmds=1]

In [29]:
prog = GCode.GCode()


def line(points=default_line, feed=300, power=150, dynamic_power=True):
    prog.G0(X=points[:, 0][0], Y=points[:, 0][1])
    prog.G1(X=points[:, 1][0], Y=points[:, 1][1])


line()
prog


<GCode>[cmds=2]

In [30]:
prog = GCode.GCode()


def line(points=default_line, feed=300, power=150, dynamic_power=True):
    prog.G0(X=points[:, 0][0], Y=points[:, 0][1])

    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M4(S=power)

    for row_idx in range(1, points.shape[0]):
        prog.G1(X=points[row_idx, 0], Y=points[row_idx, 1])
    prog.M5()


line()
prog


<GCode>[cmds=4]

Draw a 10mm x 10mm square

In [31]:
square_static = np.array([[0, 0], [10, 0], [10, 10], [0, 10], [0, 0]])


In [49]:
prog = GCode.GCode()


def line(points=default_line, feed=300, power=150, dynamic_power=True):
    prog.G0(X=points[0, 0], Y=points[0, 1])

    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M4(S=power)

    for row_idx in range(1, points.shape[0]):
        prog.G1(X=points[row_idx, 0], Y=points[row_idx, 1])
    prog.M5()


line(points=square_static)
prog


<GCode>[cmds=7]

In [50]:
for r in range(N_BrickRows):
    print(H_Block_Brick * r)


0.0
17.8
35.6
53.400000000000006
71.2


In [59]:
prog = GCode.GCode()
for r in range(1, N_BrickRows):
    y_ = np.round(r * H_Block_Brick, 4)
    row_line = np.array([[0, y_], [BlockLength, y_]])
    line(points=row_line)
prog


<GCode>[cmds=16]

In [60]:
cnc.run(prog)


^C


352.22867918014526

In [63]:
cnc.reset()


In [71]:
cnc.home()
for _ in range(360):
    if len(cnc.cmd("")) == 2:
        break
    sleep(1)


In [72]:
_


10

In [73]:
init()


<GCode>[cmds=7]

In [74]:
cnc.run(init())


0.7270307540893555

In [75]:
cnc.run(prog)


209.87263464927673